In [ ]:

######
                filters={"$or":[
                            {"tags": {"$in": ["src.models.common.Unclick_CTR_IPW_Loss"]}},
                            {"tags": {"$in": ["src.models.common.Click_CTR_IPW_Loss"]}},
                            {"tags": {"$in": ["src.models.common.Impression_CTR_IPW_Loss"]}},
                            # 这一行相当于加baseline对比
                            {"$and":[
                            {"tags": {"$in": ["src.models.common.Basic_Loss"]}},
                            {"tags": {"$in": ["src.models.descm.DESCM_Embedding"]}}]
                             }   
                            ]
                         }

#######
                filters={"$or":[
                            {"tags": {"$in": ["click_to_unclick"]}},
                            {"tags": {"$in": [""]}},
                            {"tags": {"$in": ["src.models.common.Impression_CTR_IPW_Loss"]}},
                            ],
                         "$and":[
                            {"tags": {"$in": ["src.models.descm_cycada.DESCM_Embedding_DA_cycada"]}},]
                         }

In [19]:
import wandb
import pandas as pd
api = wandb.Api()

# run = api.run("<entity>/<project>/<run_id>")
# run.config["key"] = updated_value
# run.update()
# api.runs("project_path_here", filters={"createdAt": {"$in": ["date"]}})
runs = api.runs("q191106702/CVR prediction test", 
                filters={"$or":[
                            # {"tags": {"$in": ["src.models.descm_res.DESCM_Embedding_Res_Simple"]}},
                            # {"tags": {"$in": ["src.models.descm_res.DESCM_Embedding_Res_All"]}},
                            # {"tags": {"$in": ["3-epoch"]}},
                            # {"tags": {"$in": ["2-epoch"]}},
                            # {"$and":[
                            #     {"tags": {"$in": ["src.models.descm_cycada.MultiTaskLitModel_cycada"]}},
                            #     {"tags": {"$in": ["detach_embedding"]}},
                            #         ]},        
                            {"$and":[
                                {"tags": {"$in": ["src.models.descm_res.Basic_Loss_Res_1"]}},
                                {"tags": {"$in": ["3-epoch"]}},
                            ]},        
                            {"$and":[
                                {"tags": {"$in": ["src.models.descm_res.DESCM_Embedding_Res_Simple"]}},
                                {"tags": {"$in": ["3-epoch"]}},
                            ]},
                            {"$and":[
                                {"tags": {"$in": ["src.models.descm.DESCM_Embedding"]}},
                                {"tags": {"$in": ["src.models.common.Basic_Loss"]}},
                            ]},
                            {"$and":[
                                {"tags": {"$in": ["src.models.descm.ESCM"]}},
                                {"tags": {"$in": ["src.models.common.IPW_Loss"]}},
                            ]},
                            {"$and":[
                                {"tags": {"$in": ["src.models.descm.ESCM"]}},
                                {"tags": {"$in": ["src.models.common.Basic_Loss"]}},
                            ]}
                            ]
                         }
                )

# Assume runs is your list of runs
data_list = [
    {'test/cvr_auc': run.summary._json_dict.get('test/cvr_auc', None),
    'test/ctcvr_auc': run.summary._json_dict.get('test/ctcvr_auc', None),
    'test/ctr_auc': run.summary._json_dict.get('test/ctr_auc', None),
    'tags': run.tags} for run in runs]

# Convert list of dictionaries to DataFrame
df = pd.DataFrame(data_list)
df = df.round(3)

# Function to extract tag from list
def extract_tag(row):
    for tag in tags_to_extract:
        if tag in row['tags']:
            return tag
    return None

def extract_tag_as_new_column(df, tag_to_extract, tag_name='data'):
    df[tag_name] = df.apply(extract_tag, axis=1)
    df['tags'] = [[tag for tag in tags if tag not in tags_to_extract] for tags in df['tags']]
    return df
    
    

# set data column
tags_to_extract = ['fr', 'es', 'nl', 'us', 'ccp']
df = extract_tag_as_new_column(df, tags_to_extract, 'data')
#set the method column 这里前面的就会先被提取，提取了之后就会被删除
tags_to_extract = ['src.models.common.IPW_Loss', 'src.models.descm.ESCM', 'src.models.descm.DESCM_Embedding','src.models.descm_res.Basic_Loss_Res_1', 'src.models.descm_res.DESCM_Embedding_Res_Simple',]
df = extract_tag_as_new_column(df, tags_to_extract, 'methods')
df['methods'] = df['methods'].str.split('.').str[-1]


# Convert 'tags' column to string
df['tags'] = df['tags'].apply(lambda x: ','.join(x))
# Split df into df_cvr, df_ctcvr, df_ctr_auc
df_cvr = df[['test/cvr_auc', 'data', 'methods']]
df_ctcvr = df[['test/ctcvr_auc', 'data','methods']]
df_ctr = df[['test/ctr_auc', 'data', 'methods']]

# Pivot each DataFrame
df_cvr = df_cvr.groupby(['methods', 'data']).first().reset_index()
df_cvr = df_cvr.groupby(['methods', 'data']).first().reset_index()

df_cvr = df_cvr.pivot(index='methods', columns='data', values='test/cvr_auc').reset_index()
df_ctcvr = df_ctcvr.groupby(['methods', 'data']).first().reset_index()
df_ctcvr = df_ctcvr.pivot(index='methods', columns='data', values='test/ctcvr_auc').reset_index()
df_ctr = df_ctr.groupby(['methods', 'data']).first().reset_index()
df_ctr = df_ctr.pivot(index='methods', columns='data', values='test/ctr_auc').reset_index()

# save *_cvr as csv
df_cvr.to_csv('cvr.csv', index=False)
df_ctcvr.to_csv('ctcvr.csv', index=False)
df_ctr.to_csv('ctr_auc.csv', index=False)
# Merge the DataFrames on 'method'
df_combined = df_cvr.merge(df_ctcvr, on='methods', suffixes=('_cvr', '_ctcvr'))
df_combined = df_combined.merge(df_ctr, on='methods')

# df_combined.rename(columns={'ccp': 'ccp_ctr', 'fr':'fr_ctr','nl':'nl_ctr','us':'us_ctr','es':'es_ctr'}, inplace=True)
df_combined.to_csv('combined.csv', index=False)
df_combined

data,methods,ccp_cvr,es_cvr,fr_cvr,nl_cvr,us_cvr,ccp_ctcvr,es_ctcvr,fr_ctcvr,nl_ctcvr,us_ctcvr,ccp,es,fr,nl,us
0,Basic_Loss_Res_1,0.667,NaN,NaN,NaN,NaN,0.632,NaN,NaN,NaN,NaN,0.623,NaN,NaN,NaN,NaN
1,DESCM_Embedding,0.675,0.808,0.797,0.760,0.804,0.647,0.876,0.860,0.836,0.850,0.622,0.719,0.718,0.719,0.701
2,DESCM_Embedding_Res_Simple,0.676,0.819,0.810,0.782,0.815,0.627,0.882,0.860,0.847,0.859,0.625,0.724,0.718,0.720,0.698
3,ESCM,0.671,0.789,0.778,0.725,0.769,0.639,0.861,0.844,0.814,0.837,0.621,0.717,0.715,0.715,0.699
4,IPW_Loss,0.664,0.794,0.792,0.750,0.787,0.640,0.882,0.871,0.850,0.856,0.593,0.684,0.682,0.688,0.654
